In [1]:
import sys
sys.path.append('/home/jovyan/ChestXray-14')

In [2]:
import tensorflow as tf
from modules.models import Model
from modules.utils import get_dataset
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, EarlyStopping, ReduceLROnPlateau, LearningRateScheduler

2023-02-17 23:30:33.757751: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [3]:
INPUT_PATH = "/home/jovyan/ChestXray-14/dataset/ChestXray NIH"
LABELS = ['No Finding', 'Atelectasis', 'Consolidation', 'Infiltration', 'Pneumothorax', 'Edema', 'Emphysema', 'Fibrosis', 'Effusion', 'Pneumonia', 'Pleural_Thickening', 'Cardiomegaly', 'Nodule', 'Mass', 'Hernia']

In [4]:
class Dataset:
    INPUT_PATH = INPUT_PATH
    
    def get_train(self):
        filenames = tf.io.gfile.glob(f'{self.INPUT_PATH}/data/under_sampling/train/*.tfrec')
        dataset = get_dataset(filenames)
        return dataset

    def get_test(self):
        filenames = tf.io.gfile.glob(f'{self.INPUT_PATH}/data/under_sampling/test/*.tfrec')
        dataset = get_dataset(filenames)
        return dataset

## Train Model

In [5]:
import os
CURRENT_PATH = os.path.abspath("")
CURRENT_PATH

'/home/jovyan/ChestXray-14/Sampling'

In [6]:
# Learning rate
def lr_schedule(epoch, learning_rate):
    tf.summary.scalar('learning rate', data=learning_rate, step=epoch)
    return learning_rate

In [7]:
def get_callbacks(NAME, weight_option, fold_num=None):
    model_checkpoint_callback = ModelCheckpoint(f'results/models/FacalLoss/{NAME}_{weight_option}.h5', monitor='val_loss', mode='min', save_best_only=True)
    early_stop_callback = EarlyStopping(monitor='val_loss', mode="min", patience=20, verbose=1)
    reduce_lr_callback = ReduceLROnPlateau(monitor='val_loss', mode="min", factor=0.5, patience=3, verbose=1)
    lr_logging_callback = LearningRateScheduler(lr_schedule)
    
    return model_checkpoint_callback, early_stop_callback, reduce_lr_callback, lr_logging_callback

In [8]:
# Constant variables
NAME = "EfficientNetB0"
EPOCHS = 100
NUM_FOLDS = 5

In [9]:
weight_option = None # use `imagenet` or `None` only

In [10]:
# Callbacks
model_checkpoint_callback, early_stop_callback, reduce_lr_callback, lr_logging_callback = get_callbacks(NAME, weight_option)

# Path for CSV
path = os.path.join(CURRENT_PATH, "results", "history", "training_with_FacalLoss", f"{NAME}_{weight_option}")
os.makedirs(path, exist_ok=True)

# CSV Logger
csv_logger = CSVLogger(os.path.join(path, f"EfficialNetB0_None.csv"))

# Dataset
train_dataset, test_dataset = Dataset().get_train(), Dataset().get_test()

# Modeling
transfer_model = tf.keras.applications.efficientnet.EfficientNetB0(
    include_top=False, 
    weights=weight_option,
    input_shape=(224, 224, 3),
    pooling=None
)

model = Model(
    transfer_model,
)
model = model.get_model(flatten=True)
model._name = "FacalLossOnSamplingDataset"
model.summary()

2023-02-17 23:30:36.264411: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-17 23:30:38.063592: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 31725 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:31:00.0, compute capability: 8.0


Model: "FacalLossOnSamplingDataset"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb0 (Functional)  (None, 7, 7, 1280)       4049571   
                                                                 
 flatten (Flatten)           (None, 62720)             0         
                                                                 
 dense (Dense)               (None, 128)               8028288   
                                                                 
 dense_1 (Dense)             (None, 128)               16512     
                                                                 
 dense_2 (Dense)             (None, 64)                8256      
                                                                 
 dense_3 (Dense)             (None, 64)                4160      
                                                                 
 dense_4 (Dense)             (None, 15) 

In [11]:
# Visualize
history = model.fit(
    train_dataset,
    epochs=EPOCHS,
    validation_data=test_dataset,
    verbose=1, # Show Progress Bar while Traning
    callbacks=[model_checkpoint_callback, csv_logger, early_stop_callback, reduce_lr_callback, lr_logging_callback]
)

Epoch 1/100


2023-02-17 23:30:50.821343: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100
2023-02-17 23:30:52.037709: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-02-17 23:30:52.039741: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-02-17 23:30:52.039803: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2023-02-17 23:30:52.041050: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-02-17 23:30:52.041170: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2023-02-17 23:30:56.703086: I tensorflow/stream_executor/cuda/c

3060/3060 [==============================] - 685s 217ms/step - loss: 0.2832 - auc: 0.5815 - val_loss: 0.2766 - val_auc: 0.6293 - lr: 0.0010
Epoch 2/100
3060/3060 [==============================] - 658s 215ms/step - loss: 0.2757 - auc: 0.6201 - val_loss: 0.2745 - val_auc: 0.6319 - lr: 0.0010
Epoch 3/100
3060/3060 [==============================] - 654s 214ms/step - loss: 0.2742 - auc: 0.6254 - val_loss: 0.2889 - val_auc: 0.6129 - lr: 0.0010
Epoch 4/100
3060/3060 [==============================] - 657s 215ms/step - loss: 0.2713 - auc: 0.6463 - val_loss: 0.2684 - val_auc: 0.6645 - lr: 0.0010
Epoch 5/100
3060/3060 [==============================] - 654s 214ms/step - loss: 0.2694 - auc: 0.6596 - val_loss: 0.2687 - val_auc: 0.6665 - lr: 0.0010
Epoch 6/100
3060/3060 [==============================] - 656s 214ms/step - loss: 0.2657 - auc: 0.6798 - val_loss: 0.2660 - val_auc: 0.6800 - lr: 0.0010
Epoch 7/100
3060/3060 [==============================] - 655s 214ms/step - loss: 0.2636 - auc: 0.690